In [3]:
import re
import os
import shutil
import pandas as pd

# Borrar train folder

In [46]:
path = '/home/mr1142/Documents/Data/logs/fit'

In [47]:
for di in os.listdir(path):
    shutil.rmtree(os.path.join(path, di, 'train'), ignore_errors=True)

# Borrar modelos malos

In [44]:
path = '/home/mr1142/Documents/Data/models/validation_results/validation_results' + '' + '.csv'
df = pd.read_csv(path)

In [45]:
df.sort_values('dice_coef_loss')

,type,name,loss,dice_coef_loss,loss_mask,accuracy,AUC,false_positives,false_negatives
20,unet,final_renacimiento_validation_6,0.027842,0.027473,0.078634,0.977579,0.977706,72640.0,74298.0
19,uloss,final_renacimiento_validation_6,0.102747,0.028376,0.073854,0.976893,0.977792,70053.0,81382.0
18,unet,final_renacimiento_validation_5,0.030735,0.030172,0.084180,0.975373,0.975297,76188.0,85209.0
17,uloss,renacimiento_validation_4,0.119882,0.030873,0.088655,0.975076,0.976777,80748.0,80958.0
15,uloss,renacimiento_validation_3,0.119526,0.030919,0.088200,0.975120,0.976994,80190.0,81231.0
12,unet,renacimiento_validation_2,0.032413,0.031955,0.089793,0.974418,0.973736,70983.0,94992.0
10,unet,renacimiento_validation_1,0.032621,0.032179,0.092485,0.973949,0.974434,88665.0,80353.0
14,unet,renacimiento_validation_3,0.033088,0.032720,0.098321,0.973591,0.974356,92355.0,78991.0
16,unet,renacimiento_validation_4,0.034823,0.034295,0.098386,0.972277,0.971930,88000.0,91869.0
2,unet,_albumentation_validation_2.h5,0.034487,0.034487,0.095812,0.973940,0.974166,7962.0,5701.0


In [21]:
malos_ = df[['type', 'name']][df.dice_coef_loss>0.04]

In [22]:
malos_

,type,name
30,unet,patologic_albumentation_2_validation_1
33,unet,patologic_albumentation_2_validation_2


In [25]:
malos = [df['type'][i] + '_' + df['name'][i] for i in malos_.index]
malos = [re.sub('__', '_', mal) for mal in malos]
malos = [re.sub('.h5', '', mal) for mal in malos]
malos = [re.sub('patologic_', '', mal) for mal in malos]

In [26]:
malos

['unet_albumentation_2_validation_1', 'unet_albumentation_2_validation_2']

In [27]:
def match(lista_resultados, lista_match):
    return [f for f in lista_resultados if any([bool(re.search(match, f)) for match in lista_match])]

In [28]:
def eliminar(path, malos):
    files = os.listdir(path)
    files = match(files, malos)
    if len(files) == len(malos):
        for file in files:
            try:
                os.remove(os.path.join(path, file))
                print(f'file {file} removed')
            except:
                shutil.rmtree(os.path.join(path,file), ignore_errors=True)
                print(f'folder {file} removed')
    else:
        print('no tienen el mismo tamaño')
        print(files)

### Logs

In [29]:
path = '/home/mr1142/Documents/Data/logs/fit'

In [30]:
eliminar(path, malos)

folder unet_albumentation_2_validation_1 removed
folder unet_albumentation_2_validation_2 removed


### Modelos

In [31]:
path = '/home/mr1142/Documents/Data/models'

In [32]:
eliminar(path, malos)

file unet_albumentation_2_validation_1.h5 removed
file unet_albumentation_2_validation_2.h5 removed


# Borrar datos

In [35]:
path = '/home/mr1142/Documents/Data/models/validation_results'
csvs = os.listdir(path)

for csv in csvs:
    df = pd.read_csv(os.path.join(path, csv))
    index = [i for i in df.index if any([bool(re.search(mal, df['name'][i])) for mal in malos_['name']])]
    df = df.drop(index)
    df.to_csv(os.path.join(path, csv), index = False)